# [LADOT]((https://www.ladotbus.com/)) GTFS Realtime Vehicle positions

GTFS Realtime is a feed specification that allows public transportation agencies to provide realtime updates about their fleet to application developers. It is used globally by many transit agencies and is one of the widely adopted open data standards for public transportation industry.

## Imports

In [1]:
from google.transit import gtfs_realtime_pb2
from protobuf_to_dict import protobuf_to_dict
import requests
from collections import OrderedDict
import pandas as pd
import folium

## Urls

Vehicle Positions — information about the vehicles including location and congestion level in real time.

In [2]:
# gtfs_schedule_url = "https://ladotbus.com/gtfs"
ladot_vehicle_positions_url = "https://ladotbus.com/gtfs-rt/vehiclepositions"
#ladot_service_alerts_url = "https://ladotbus.com/gtfs-rt/alerts"
#ladot_trip_updates_url = "https://ladotbus.com/gtfs-rt/tripupdates"

## Request

We will download a GTFS-realtime data feed from our URL, parsing it as a [FeedMessage (the root type of the GTFS-realtime schema)](https://developers.google.com/transit/gtfs-realtime/reference), and iterating over the results. Code snippet adapted from [Google Transit APIs](https://developers.google.com/transit/gtfs-realtime/examples/python-sample)

In [3]:
# Get FeedMessage from url
feed = gtfs_realtime_pb2.FeedMessage()
response = requests.get(ladot_vehicle_positions_url)
feed.ParseFromString(response.content)

# Let's look at one entity
feed.entity[0]    

id: "vehicle_7254"
vehicle {
  trip {
    trip_id: "30-7zisxioi4"
    start_time: "15:23:00"
    start_date: "20220606"
    direction_id: 1
  }
  position {
    latitude: 34.05593490600586
    longitude: -118.26082611083984
    bearing: 207.1741943359375
    speed: 0.0
  }
  timestamp: 1654559184
  vehicle {
    id: "7254"
    label: "12303"
  }
  occupancy_status: MANY_SEATS_AVAILABLE
}

## Data

Each entity contains the following information:

- `id`
- `vehicle`
    - `trip`
        - `trip_id`
        - `start_time`
        - `start_date`
        - `direction_id`: DATETYPE, DESCRIPTION, EXAMPLES
    - `position`
        - `latitude`
        - `longitude`
        - `bearing`
        - `speed`
    - `timestamp`
    - `vehicle`
        - `id`
        - `label`
    - `occupancy_status`

## Convert feed to dataframe

We want to parse GTFS Real time Protobuf into more usable tabular format. Let's use a FOR LOOP to iterate over the nested dictionary structure and collect each rows and append & collect it in form of pandas dataframe.

In [4]:
dict_obj = protobuf_to_dict(feed)

In [12]:
collector = []

for block in dict_obj['entity']:
    try:
        row = OrderedDict()
        # id
        row['id'] = block['id']
        # vehicle blocks
        trip = block['vehicle']['trip']
        position = block['vehicle']['position']
        vehicle = block['vehicle']['vehicle']

        # trip
        row['trip_id'] = trip.get('trip_id','')
        row['start_time'] = trip.get('start_time','')
        row['start_date'] = trip.get('start_date','')
        row['direction_id'] = trip.get('direction_id','')
        # position 
        row['latitude'] = position.get('latitude','')
        row['longitude'] = position.get('longitude','')
        row['bearing'] = position.get('bearing','')
        row['speed'] = position.get('speed','')
        # timestamp
        row['timestamp'] = block['vehicle']['timestamp']
        # vehicle
        row['id'] = vehicle.get('id','')
        row['label'] = vehicle.get('label','')
        # occupancy_status
        row['occupancy_status'] = block['vehicle']['occupancy_status']

        collector.append(row)
    except:
        pass
    
df = pd.DataFrame(collector)

In [13]:
df

,id,trip_id,start_time,start_date,direction_id,latitude,longitude,bearing,speed,timestamp,label,occupancy_status
0,7254,30-7zisxioi4,15:23:00,20220606,1,34.055935,-118.260826,207.174194,0.00000,1654559184,12303,1
1,6457,30-TyyMGjYn5nF,16:25:00,20220606,0,34.036053,-118.187286,271.140045,0.44704,1654559183,09313,1
2,1641,30-IZjFYi-Out5,16:00:00,20220606,1,34.066010,-118.218117,270.000275,4.91744,1654559181,15346,0
3,1807,30-oGC8Z40CEsV,16:30:00,20220606,0,33.993141,-118.317360,89.999825,6.25856,1654559182,15373,1
4,1782,30-ulusa0ernT-,16:05:00,20220606,0,34.069271,-118.190376,68.878128,4.91744,1654559181,15356,0
...,...,...,...,...,...,...,...,...,...,...,...,...
180,5822,183-8eba07jrs,16:51:00,20220606,0,34.028973,-118.269020,4.000000,9.83488,1654559184,13315,0
181,6229,183-8idvm15h0,16:20:00,20220606,0,34.034260,-118.267410,211.000000,3.57632,1654559182,20306,0
182,5832,183-6o7jt5boc,16:16:00,20220606,0,34.046795,-118.255898,127.168106,0.44704,1654559183,15330,1
183,6238,183-quhzfmkkl,15:42:00,20220606,0,34.050816,-118.258492,124.000000,4.91744,1654559183,20302,0


## Visualization using folium

In [25]:
veh_map = folium.Map(prefer_canvas=True)

def plotDot(point):
    '''input: series that contains a numeric named latitude and a numeric named longitude
    this function creates a CircleMarker and adds it to veh_map'''
    html = f"""<b>ID:</b> {point.id}<br>
    <b>Trip ID:</b> {point.trip_id}<br>
    <b>Direction ID:</b> {point.direction_id}<br>
    """
    
    iframe = folium.IFrame(html, width=150, height=100)
    
    folium.CircleMarker(location=[point.latitude, point.longitude],
                        popup=folium.Popup(iframe),
                        radius=5,
                        weight=5).add_to(veh_map)

#use df.apply(,axis=1) to "iterate" through every row in your dataframe
df.apply(plotDot, axis = 1)


#Set the zoom to the maximum possible
veh_map.fit_bounds(veh_map.get_bounds())

#Save the map to an HTML file
veh_map.save('html_map_output/veh_map.html')

veh_map